In [1]:
# Import the required libraries.
import re
import math
import random
import collections
import operator
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter

from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras import Model, Sequential
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import *
from sklearn.metrics import classification_report

random.seed(11)
np.random.seed(11)

Using TensorFlow backend.


In [2]:
!ls

 NER-Dataset-10Types-Train.txt	 NER-TestSet-10Types-RNN-Predictions.txt
 NER-Dataset--TestSet.txt	'Q2 - NER Prediction - 10 Types.ipynb'
 NER-Dataset-Train.txt		'Q2  - NER Prediction.ipynb'


In [3]:
with open('NER-Dataset-Train.txt', 'r') as f:
    ner_dataset = f.readlines()

In [4]:
sentences = []
words = []
tags = []
for line in ner_dataset:
    line = line.strip()
    if line == '':
        sentences.append((words, tags))
        words = []
        tags = []
    else:
        word, tag = line.split('\t')
        words.append(word)
        tags.append(tag)

if len(words) > 0:
    sentences.append((words, tags))
    words = []
    tags= []

In [5]:
vocab_counts = Counter(sum([a[0] for a in sentences], [])).most_common()
words_to_keep = set([word for word, count in vocab_counts if count > 1])

In [6]:
with open('NER-Dataset--TestSet.txt', 'r') as f:
    test_dataset = f.readlines()

test_sentences = []
words = []
for line in test_dataset:
    line = line.strip()
    if line == '':
        test_sentences.append((words,))
        words = []
    else:
        word = line
        words.append(word)

if len(words) > 0:
    test_sentences.append((words,))
    words = []

In [7]:
word_features = ['twoDigitNum',
                'fourDigitNum',
                'containsDigitAndAlpha',
                'containsDigitAndDash',
                'containsDigitAndSlash',
                'containsDigitAndComma',
                'containsDigitAndPeriod',
                'otherNum',
                'allCaps',
                'capPeriod',
                'firstWord',
                'initCap',
                'lowerCase',
                'other']

In [8]:
def get_word_features(sentence):
    features = []
    ## Optimize and use an Enum!
    firstword = True
    for word in sentence:
        if word.isnumeric() and len(word) == 2:
            features.append('twoDigitNum')
        elif word.isnumeric() and len(word) == 4:
            features.append('fourDigitNum')
        elif word.isalnum() and not word.isalpha() and not word.isnumeric():
            features.append('containsDigitAndAlpha')
        elif word.replace('-', '').isnumeric():
            features.append('containsDigitAndAlpha')
        elif word.replace('/', '').isnumeric():
            features.append('containsDigitAndSlash')
        elif word.replace('.', '').replace(',', '').isnumeric() and ',' in word:
            features.append('containsDigitAndComma')
        elif word.replace('.', '').isnumeric():
            features.append('containsDigitAndPeriod')
        elif word.isnumeric():
            features.append('otherNum')
        elif word.isupper():
            features.append('allCaps')
        elif len(word) == 2 and word[0].isupper() and word[1] == '.':
            features.append('capPeriod')
        elif firstword:
            features.append('firstWord')
        elif word[0].isupper():
            features.append('initCap')
        elif word.islower():
            features.append('lowerCase')
        else:
            features.append('other')
        firstword = False

    return features

In [9]:
max_len_found = max(len(s[0]) for s in sentences)
max_len = max_len_found + ((50 - (max_len_found % 50)) % 50)

In [10]:
eye_mat = list(np.eye(len(word_features)))
wordfeat2float = {feat: eye_mat[i] for i, feat in enumerate(word_features)}

In [11]:
word2idx = {'UNK': 0, 'PAD': 1}
word2idx.update({word: i + 2 for i, word in enumerate(sorted(words_to_keep))})

In [12]:
def numberize_sentence(words, max_len=50):
    features = get_word_features(words)
    word_idx = [word2idx[w] if w in word2idx.keys() else word2idx['UNK'] for w in words]
    feat_np = [wordfeat2float[f] for f in features]
    word_padding = [word2idx['PAD'] for _ in range(max_len - len(word_idx))]
    feat_padding = [np.ones((len(word_features),)) * 2 for _ in range(max_len - len(word_idx))]
    word_idx = np.asarray(word_idx + word_padding)
    feat_np = np.asarray(feat_np + feat_padding)
    return word_idx, feat_np

In [13]:
labels = set.union(*(set(s[1]) for s in sentences))
idx2labels = {i: s for i, s in enumerate(labels)}
n_labels = len(labels)
eye_mat = list(np.eye(len(labels)))
labels2float = {feat: eye_mat[i] for i, feat in enumerate(labels)}

def numberize_labels(gt_labels, max_len=50):
    labels_np = [labels2float[l] for l in gt_labels]
    labels_padding = [labels2float['O'] for _ in range(max_len - len(gt_labels))]
    return np.asarray(labels_np + labels_padding)

In [14]:
def create_model():
    input_words = Input(shape = (max_len,))
    input_feats = Input(shape = (max_len, len(word_features)))
    masked_words = Masking(mask_value = 1)(input_words)
    masked_feats = Masking(mask_value = 2)(input_feats)
    emb = Embedding(input_dim = (len(word2idx)), output_dim = 50, input_length = max_len)(masked_words)
    drop_emb = Dropout(0.1)(emb)
    concat_out = Concatenate()([drop_emb, masked_feats])
    rnn_out = Bidirectional(SimpleRNN(units = 100, return_sequences = True, recurrent_dropout = 0.1))(concat_out)
    dense_out = TimeDistributed(Dense(n_labels, activation = "softmax"))(rnn_out)
    model = Model(inputs = [input_words, input_feats], outputs = dense_out)
    model.summary()
    return model

In [15]:
parsed_sentences = [(numberize_sentence(s[0]), numberize_labels(s[1])) for s in sentences]
parsed_test_sentences = [numberize_sentence(s[0]) for s in test_sentences]

In [16]:
Counter(sum([s[1] for s in sentences], []))
Counter(sum([np.argmax(s[1], axis=-1).tolist() for s in parsed_sentences], []))

Counter({1: 44028, 2: 582, 0: 390})

In [17]:
# Build the test and training sets of sentences.
kf = KFold(n_splits = 5, shuffle = False)
parsed_sentences = np.asarray(parsed_sentences)
scores = []
scores1 = []
y_pred_idx = []
y_pred_idx1 = []
y_test_idx = []
y_test_idx1 = []

preds = []

for train_index, test_index in kf.split(parsed_sentences):
    train_data = parsed_sentences[train_index]
    test_data = parsed_sentences[test_index]
    X_train = [np.asarray([a[0][0] for a in train_data]), np.asarray([a[0][1] for a in train_data])]
    Y_train = np.asarray([a[1] for a in train_data])
    X_test = [np.asarray([a[0][0] for a in test_data]), np.asarray([a[0][1] for a in test_data])]
    Y_test = np.asarray([a[1] for a in test_data])
    model = create_model()
    model.compile(optimizer = 'rmsprop',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

    model.fit(X_train, Y_train, epochs = 3, validation_split = 0.2, batch_size = 4)

    y_pred_padded = np.argmax(model.predict(X_test), axis = -1)
    y_true_padded = np.argmax(Y_test, axis = -1)
    
    for i in range(X_test[0].shape[0]):
        for j in range(X_test[0].shape[1]):
            if X_test[0][i][j] == word2idx['PAD']:
                continue
            else:
                pred = y_pred_padded[i][j]
                true = y_true_padded[i][j]
                y_pred_idx.append(pred)
                y_test_idx.append(true)
                scores.append(pred == true)

    prec_, rec_, fscore_, _ = precision_recall_fscore_support(y_test_idx, y_pred_idx, average = 'macro')
    print('[still updating...] Accuracy: {}, Precision: {}, Recall: {}, FScore: {}'.format(np.asarray(scores).mean(), prec_, rec_, fscore_))
    
prec, rec, fscore, _ = precision_recall_fscore_support(y_test_idx, y_pred_idx, average = 'macro')
print('Accuracy: {}, Precision: {}, Recall: {}, FScore: {}'.format(np.asarray(scores).mean(), prec, rec, fscore))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 50)           0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 50)       72900       masking_1[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50, 14)       0                                            
____________________________________________________________________________________________

/home/shikhar/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 576 samples, validate on 144 samples
Epoch 1/3
576/576 [==============================] - 15s 26ms/step - loss: 0.0950 - accuracy: 0.9346 - val_loss: 0.0667 - val_accuracy: 0.9485
Epoch 2/3
576/576 [==============================] - 14s 24ms/step - loss: 0.0619 - accuracy: 0.9475 - val_loss: 0.0603 - val_accuracy: 0.9506
Epoch 3/3
576/576 [==============================] - 13s 22ms/step - loss: 0.0512 - accuracy: 0.9556 - val_loss: 0.0615 - val_accuracy: 0.9478
[still updating...] Accuracy: 0.949859943977591, Precision: 0.6654204992439212, Recall: 0.5902111321273259, FScore: 0.6223229328172685
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50)           0                                            
____________________________________________________________________________

/home/shikhar/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 576 samples, validate on 144 samples
Epoch 1/3
576/576 [==============================] - 17s 29ms/step - loss: 0.0895 - accuracy: 0.9420 - val_loss: 0.0694 - val_accuracy: 0.9435
Epoch 2/3
576/576 [==============================] - 18s 31ms/step - loss: 0.0627 - accuracy: 0.9481 - val_loss: 0.0582 - val_accuracy: 0.9535
Epoch 3/3
576/576 [==============================] - 18s 31ms/step - loss: 0.0522 - accuracy: 0.9573 - val_loss: 0.0567 - val_accuracy: 0.9528
[still updating...] Accuracy: 0.9489781536293164, Precision: 0.7275867773129755, Recall: 0.5541793045701331, FScore: 0.6144008766626959
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 50)           0                                            
___________________________________________________________________________

...

Train on 576 samples, validate on 144 samples
Epoch 1/3
576/576 [==============================] - 16s 28ms/step - loss: 0.0897 - accuracy: 0.9382 - val_loss: 0.0738 - val_accuracy: 0.9446
Epoch 2/3
576/576 [==============================] - 23s 40ms/step - loss: 0.0620 - accuracy: 0.9502 - val_loss: 0.0645 - val_accuracy: 0.9410
Epoch 3/3
576/576 [==============================] - 26s 44ms/step - loss: 0.0512 - accuracy: 0.9570 - val_loss: 0.0575 - val_accuracy: 0.9546
[still updating...] Accuracy: 0.9501140250855188, Precision: 0.7463736340760093, Recall: 0.5559133901490454, FScore: 0.620367438437964
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 50)           0                                            
____________________________________________________________________________

/home/shikhar/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 576 samples, validate on 144 samples
Epoch 1/3
576/576 [==============================] - 24s 42ms/step - loss: 0.0959 - accuracy: 0.9334 - val_loss: 0.0664 - val_accuracy: 0.9499
Epoch 2/3
576/576 [==============================] - 24s 42ms/step - loss: 0.0672 - accuracy: 0.9426 - val_loss: 0.0723 - val_accuracy: 0.9374
Epoch 3/3
576/576 [==============================] - 24s 42ms/step - loss: 0.0555 - accuracy: 0.9519 - val_loss: 0.0595 - val_accuracy: 0.9485
[still updating...] Accuracy: 0.9518587095855291, Precision: 0.7446822750496102, Recall: 0.530571870576986, FScore: 0.5972842935397518
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 50)           0                                            
____________________________________________________________________________

/home/shikhar/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 576 samples, validate on 144 samples
Epoch 1/3
576/576 [==============================] - 24s 42ms/step - loss: 0.1032 - accuracy: 0.9293 - val_loss: 0.0574 - val_accuracy: 0.9558
Epoch 2/3
576/576 [==============================] - 22s 39ms/step - loss: 0.0660 - accuracy: 0.9438 - val_loss: 0.0433 - val_accuracy: 0.9586
Epoch 3/3
576/576 [==============================] - 24s 41ms/step - loss: 0.0555 - accuracy: 0.9522 - val_loss: 0.0433 - val_accuracy: 0.9589
[still updating...] Accuracy: 0.950858123569794, Precision: 0.7194213949530299, Recall: 0.5185923851796569, FScore: 0.5811032772504607
Accuracy: 0.950858123569794, Precision: 0.7194213949530299, Recall: 0.5185923851796569, FScore: 0.5811032772504607


In [18]:
X_test_data = [np.asarray([a[0] for a in parsed_test_sentences]), np.asarray([a[1] for a in parsed_test_sentences])]
predictions_full = model.predict(X_test_data)

In [19]:
predictions_list = []
for i, s in enumerate(test_sentences):
    output = []
    for j, w in enumerate(s[0]):
        output.append(np.argmax(predictions_full[i][j]))
    predictions_list.append(output)

In [20]:
with open('NER-TestSet-RNN-Predictions.txt', 'w', encoding = 'utf-8') as f:
    for words, predictions in zip(test_sentences, predictions_list):
        assert(len(words[0]) == len(predictions))
        for word, prediction in zip(words[0], predictions):
            f.writelines(word + '\t' + idx2labels[prediction] + '\n')
        f.writelines('\n')